In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D # type: ignore
from tensorflow.keras import Model # type: ignore

class CheckersAI(Model):
    def __init__(self):
        super(CheckersAI, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(64, activation='relu')
        self.d2 = Dense(64, activation='relu')
        self.d3 = Dense(64, activation='relu')
        self.out = Dense(64, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        x = self.d3(x)
        return self.out(x)

POPULATION_SIZE = 50
NUM_GENERATIONS = 100
MUTATION_RATE = 0.1

def initialize_population():
    return [CheckersAI() for _ in range(POPULATION_SIZE)]

def evaluate_fitness(player):
    return np.random.randint(50, 100)  # Random fitness score for demonstration

def tournament_selection(population, fitness_scores, tournament_size=5):
    selected_players = []
    for _ in range(POPULATION_SIZE):
        participants = np.random.choice(len(population), tournament_size, replace=False)
        winner_index = np.argmax([fitness_scores[i] for i in participants])
        selected_players.append(population[participants[winner_index]])
    return selected_players

def crossover(parent1, parent2):
    child1 = CheckersAI()
    child2 = CheckersAI()

    for i in range(len(child1.trainable_variables)):
        mask = np.random.choice([True, False], size=parent1.trainable_variables[i].shape)
        child1.trainable_variables[i] = tf.where(mask, parent1.trainable_variables[i], parent2.trainable_variables[i])
        child2.trainable_variables[i] = tf.where(mask, parent2.trainable_variables[i], parent1.trainable_variables[i])

    return child1, child2

def mutate(player, mutation_rate=0.1):
    mutated_player = CheckersAI()

    for i in range(len(mutated_player.trainable_variables)):
        if np.random.rand() < mutation_rate:
            noise = np.random.normal(scale=0.1, size=player.trainable_variables[i].shape)
            mutated_player.trainable_variables[i] = player.trainable_variables[i] + noise
        else:
            mutated_player.trainable_variables[i] = player.trainable_variables[i]

    return mutated_player

def evolve_population(population):
    fitness_scores = [evaluate_fitness(player) for player in population]
    selected_players = tournament_selection(population, fitness_scores)
    next_generation = []
    for _ in range(POPULATION_SIZE // 2):
        parent1 = selected_players[np.random.randint(len(selected_players))]
        parent2 = selected_players[np.random.randint(len(selected_players))]
        child1, child2 = crossover(parent1, parent2)
        next_generation.extend([mutate(child1), mutate(child2)])
    return next_generation

class HumanPlayer:
    def __init__(self, color):
        self.color = color

    def make_move(self, board):
        pass

class NeuralNetworkPlayer:
    def __init__(self, color, model, game):
        self.color = color
        self.model = model
        self.game = game
        self.max_attempts = 5

    def make_move(self, board, attempt=1):
        if attempt > self.max_attempts:
            print("AI reached maximum retry attempts. Skipping turn.")
            self.game.update()  # Update game state without making a valid move
            return

        # Get the current state of the board
        board_state = self.get_board_state(board)

        # Reshape the board state to match the input shape of the neural network
        input_state = np.expand_dims(board_state, axis=0)
        input_state = input_state.reshape((1, 8, 8, 1))

        # Get the predicted probabilities for each possible move
        move_probabilities = self.model.predict(input_state)

        # Verify the input state (print for debugging)
        print("Input State Shape:", input_state.shape)
        print("Input State:")
        print(input_state)

        # Print the move probabilities
        print("Move Probabilities:", move_probabilities)

        # Find the move with the highest probability
        best_move_index = np.argmax(move_probabilities)
        best_move_row = best_move_index // COLS
        best_move_col = best_move_index % COLS

        # Attempt to make the selected move
        if board.select(best_move_row, best_move_col):  # Ensure the move is valid
            self.game.update()
        else:
            print(f"Attempt {attempt}: AI made an invalid move.")
            self.make_move(board, attempt + 1)

        # Update the game state after the move
        self.game.update()



    def get_board_state(self, board):
        board_state = np.zeros((8, 8))
        for row in range(8):
            for col in range(8):
                piece = board.get_piece(row, col)
                if piece != 0:
                    if piece.color == 255:
                        board_state[row][col] = 1
                    else:
                        board_state[row][col] = -1
        return board_state.flatten()
import pygame

WIDTH, HEIGHT = 800, 800
ROWS, COLS = 8, 8
SQUARE_SIZE = WIDTH // COLS

RED = (255, 0, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
GREY = (128, 128, 128)

# CROWN_IMG = pygame.transform.scale(pygame.image.load('G:\\frontend\\assingment_2\\assignment3\\EA\\assets\\crown.png'), (44, 25))

class Piece:
    PADDING = 15
    OUTLINE = 2

    def __init__(self, row, col, color):  # Corrected method name
        self.row = row
        self.col = col
        self.color = color
        self.king = False
        self.calc_pos()

    def calc_pos(self):
        self.x = SQUARE_SIZE * self.col + SQUARE_SIZE // 2
        self.y = SQUARE_SIZE * self.row + SQUARE_SIZE // 2

    def make_king(self):
        self.king = True
    
    def draw(self, win):
        radius = SQUARE_SIZE // 2 - self.PADDING
        pygame.draw.circle(win, GREY, (self.x, self.y), radius + self.OUTLINE)
        pygame.draw.circle(win, self.color, (self.x, self.y), radius)
        # if self.king:
        #     win.blit(CROWN_IMG, (self.x - CROWN_IMG.get_width() // 2, self.y - CROWN_IMG.get_height() // 2))

    def move(self, row, col):
        self.row = row
        self.col = col
        self.calc_pos()

class Board:
    def __init__(self):  
        self.board = []
        self.red_left = self.white_left = 12
        self.red_kings = self.white_kings = 0
        self.create_board()
    
    def draw_squares(self, win):
        win.fill(BLACK)
        for row in range(ROWS):
            for col in range(row % 2, COLS, 2):
                pygame.draw.rect(win, WHITE, (row * SQUARE_SIZE, col * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))

    def move(self, piece, row, col):
        self.board[piece.row][piece.col], self.board[row][col] = self.board[row][col], self.board[piece.row][piece.col]
        piece.move(row, col)

        if row == ROWS - 1 or row == 0:
            piece.make_king()
            if piece.color == WHITE:
                self.white_kings += 1
            else:
                self.red_kings += 1 

    def get_piece(self, row, col):
        return self.board[row][col]

    def create_board(self):
        for row in range(ROWS):
            self.board.append([])
            for col in range(COLS):
                if col % 2 == ((row + 1) % 2):
                    if row < 3:
                        self.board[row].append(Piece(row, col, BLUE))
                    elif row > 4:
                        self.board[row].append(Piece(row, col, RED))
                    else:
                        self.board[row].append(0)
                else:
                    self.board[row].append(0)
        
    def draw(self, win):
        self.draw_squares(win)
        for row in range(ROWS):
            for col in range(COLS):
                piece = self.board[row][col]
                if piece != 0:
                    piece.draw(win)

    def remove(self, pieces):
        for piece in pieces:
            self.board[piece.row][piece.col] = 0
            if piece != 0:
                if piece.color == RED:
                    self.red_left -= 1
                else:
                    self.white_left -= 1
    
    def winner(self):
        if self.red_left <= 0:
            return WHITE
        elif self.white_left <= 0:
            return RED
        
        return None 
    
    def get_valid_moves(self, piece):
        moves = {}
        left = piece.col - 1
        right = piece.col + 1
        row = piece.row

        if piece.color == RED or piece.king:
            moves.update(self._traverse_left(row -1, max(row-3, -1), -1, piece.color, left))
            moves.update(self._traverse_right(row -1, max(row-3, -1), -1, piece.color, right))
        if piece.color == BLUE or piece.king:
            moves.update(self._traverse_left(row +1, min(row+3, ROWS), 1, piece.color, left))
            moves.update(self._traverse_right(row +1, min(row+3, ROWS), 1, piece.color, right))
    
        return moves

    def _traverse_left(self, start, stop, step, color, left, skipped=[]):
        moves = {}
        last = []
        for r in range(start, stop, step):
            if left < 0:
                break
            
            current = self.board[r][left]
            if current == 0:
                if skipped and not last:
                    break
                elif skipped:
                    moves[(r, left)] = last + skipped
                else:
                    moves[(r, left)] = last
                
                if last:
                    if step == -1:
                        row = max(r-3, 0)
                    else:
                        row = min(r+3, ROWS)
                    moves.update(self._traverse_left(r+step, row, step, color, left-1,skipped=last))
                    moves.update(self._traverse_right(r+step, row, step, color, left+1,skipped=last))
                break
            elif current.color == color:
                break
            else:
                last = [current]

            left -= 1
        
        return moves

    def _traverse_right(self, start, stop, step, color, right, skipped=[]):
        moves = {}
        last = []
        for r in range(start, stop, step):
            if right >= COLS:
                break
            
            current = self.board[r][right]
            if current == 0:
                if skipped and not last:
                    break
                elif skipped:
                    moves[(r,right)] = last + skipped
                else:
                    moves[(r, right)] = last
                
                if last:
                    if step == -1:
                        row = max(r-3, 0)
                    else:
                        row = min(r+3, ROWS)
                    moves.update(self._traverse_left(r+step, row, step, color, right-1,skipped=last))
                    moves.update(self._traverse_right(r+step, row, step, color, right+1,skipped=last))
                break
            elif current.color == color:
                break
            else:
                last = [current]

            right += 1
        
        return moves

    def select(self, row, col):
        piece = self.get_piece(row, col)
        if piece != 0 and piece.color == RED:
            self.selected = piece
            self.valid_moves = self.get_valid_moves(piece)
            return True
        else:
            return False

    def move_selected_piece(self, new_row, new_col):
        if (new_row, new_col) in self.valid_moves:
            self.move(self.selected, new_row, new_col)
            self.selected = None
            return True
        else:
            return False

class Game:
    def __init__(self, win, player1=None, player2=None):
        self._init()
        self.win = win
        population = initialize_population()
        best_player = max(population, key=evaluate_fitness)
        self.player1 = player1 if player1 else HumanPlayer(RED)
        self.player2 = player2 if player2 else NeuralNetworkPlayer(WHITE, best_player, self)  # Pass the game object
        self.current_player = self.player1  # Start with player1
        self.player2.game = self # Start with player1
        self.player2.model = best_player  # Assign model to player

    def winner(self):
        return self.board.winner()

    def _init(self):
        self.selected = None
        self.board = Board()
        self.turn = RED
        self.valid_moves = {}

    def select(self, row, col):
        if self.selected:
            result = self._move(row, col)
            if not result:
                self.selected = None
                self.select(row, col)

        piece = self.board.get_piece(row, col)
        if piece != 0 and piece.color == self.turn:
            self.selected = piece
            self.valid_moves = self.board.get_valid_moves(piece)
            return True

        return False

    def _move(self, row, col):
        piece = self.board.get_piece(row, col)
        if self.selected and piece == 0 and (row, col) in self.valid_moves:
            self.board.move(self.selected, row, col)
            skipped = self.valid_moves[(row, col)]
            if skipped:
                self.board.remove(skipped)
            self.change_turn()
            return True

        return False

    def change_turn(self):
        self.valid_moves = {}
        self.current_player = self.player2 if self.current_player == self.player1 else self.player1
        # if self.current_player == self.player1:
        #     self.current_player = self.player2
        # else:
        #     self.current_player = self.player1

        # Clear the selected piece after each turn
        self.selected = None

        # Trigger AI player's move
        self.current_player.make_move(self.board)  # Ensure the current player makes a move
        self.update()
 
    def update(self):
        self.board.draw(self.win)
        self.draw_valid_moves(self.valid_moves)
        pygame.display.update()

    def draw_valid_moves(self, moves):
        for move in moves:
            row, col = move
            pygame.draw.circle(self.win, BLUE, (col * SQUARE_SIZE + SQUARE_SIZE // 2, row * SQUARE_SIZE + SQUARE_SIZE // 2), 15)

def main():
    pygame.init()
    win = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Checkers")

    population = initialize_population()
    best_player = max(population, key=evaluate_fitness)
    game = None  # Define game variable outside the loop

    game = Game(win, player2=NeuralNetworkPlayer(WHITE, best_player, game))  # Pass the game object
    clock = pygame.time.Clock()

    while True:
        clock.tick(60)

        if game.winner() is not None:
            print(game.winner())
            break

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()

            if event.type == pygame.MOUSEBUTTONDOWN:
                pos = pygame.mouse.get_pos()
                row, col = pos[1] // SQUARE_SIZE, pos[0] // SQUARE_SIZE
                game.select(row, col)

        game.update()
        # Debugging: Print board state and AI move probabilities
        if isinstance(game.current_player, NeuralNetworkPlayer):
            current_player = game.current_player
            board_state = current_player.get_board_state(game.board)
            input_state = np.expand_dims(board_state, axis=0)
            input_state = input_state.reshape((1, 8, 8, 1))
            move_probabilities = current_player.model.predict(input_state)
            print("Move Probabilities:", move_probabilities)

    pygame.quit()

if __name__ == "__main__":
    main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
Input State Shape: (1, 8, 8, 1)
Input State:
[[[[ 0.]
   [-1.]
   [ 0.]
   [-1.]
   [ 0.]
   [-1.]
   [ 0.]
   [-1.]]

  [[-1.]
   [ 0.]
   [-1.]
   [ 0.]
   [-1.]
   [ 0.]
   [-1.]
   [ 0.]]

  [[ 0.]
   [-1.]
   [ 0.]
   [-1.]
   [ 0.]
   [-1.]
   [ 0.]
   [-1.]]

  [[ 0.]
   [ 0.]
   [ 0.]
   [ 0.]
   [ 0.]
   [ 0.]
   [ 0.]
   [ 0.]]

  [[ 0.]
   [ 0.]
   [ 0.]
   [ 0.]
   [ 0.]
   [-1.]
   [ 0.]
   [ 0.]]

  [[-1.]
   [ 0.]
   [-1.]
   [ 0.]
   [ 0.]
   [ 0.]
   [-1.]
   [ 0.]]

  [[ 0.]
   [-1.]
   [ 0.]
   [-1.]
   [ 0.]
   [-1.]
   [ 0.]
   [-1.]]

  [[-1.]
   [ 0.]
   [-1.]
   [ 0.]
   [-1.]
   [ 0.]
   [-1.]
   [ 0.]]]]
Move Probabilities: [[0.01563869 0.01533675 0.01620332 0.01542218 0.01574495 0.01544723
  0.01497553 0.01519132 0.01534169 0.01523994 0.01520971 0.0162567
  0.01604271 0.01535487 0.01447975 0.01596392 0.01512249 0.01555216
  0.01611275 0.01627757 0.01459475 0.01630251 0.01671465 0.01526006
  0.01529104 0.01598185 0.016699

error: display Surface quit